In [ ]:
# ✅ Environment Setup
import os
import pandas as pd
from dotenv import load_dotenv
from sqlalchemy import create_engine
from langchain_openai import ChatOpenAI
from langchain_experimental.agents.agent_toolkits import create_csv_agent
from langchain.sql_database import SQLDatabase
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.agents import create_sql_agent, AgentType
from langchain.memory import ConversationBufferMemory


In [ ]:
# 🔐 Load API Keys
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

if not OPENAI_API_KEY:
    raise ValueError("❌ OPENAI_API_KEY is not loaded from .env")

print("✅ API key successfully loaded")

In [ ]:
csv_path = "/Users/keshavsaraogi/Desktop/indorama/eureka-data/clean-csv/cleaned_finance_packaging.csv"
df = pd.read_csv(csv_path)

In [ ]:
llm = ChatOpenAI(
    model="gpt-4o",
    temperature=0.5
)

In [ ]:
finance_csv_memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
finance_sql_memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

In [ ]:
# 🤖 CSV Agent
csv_agent = create_csv_agent(
    llm,
    csv_path,
    verbose=True,
    allow_dangerous_code=True,
    handle_parsing_errors=True,
    max_iterations=20,
    max_execution_time=60,
    agent_type=AgentType.OPENAI_FUNCTIONS,
    memory=finance_csv_memory
)

In [ ]:
# 🛢️ SQL Agent
engine = create_engine("sqlite:///finance_packaging.db")
df.to_sql("finance_packaging_data", engine, index=False, if_exists="replace")
db = SQLDatabase(engine)

sql_agent = create_sql_agent(
    llm,
    toolkit=SQLDatabaseToolkit(db=db, llm=llm),
    verbose=True,
    handle_parsing_errors=True,
    max_iterations=20,
    max_execution_time=60,
    agent_type=AgentType.OPENAI_FUNCTIONS,
    memory=finance_sql_memory
)


In [ ]:
# 🧭 Routing Function
def route_query(query: str):
    query = query.lower()
    if any(keyword in query for keyword in ["sum", "avg", "group", "filter", "division", "unit price", "balance"]):
        return "sql"
    else:
        return "csv"

# Master Agent
def master_agent(query: str):
    source = route_query(query)
    if source == "sql":
        return sql_agent.run(query)
    else:
        return csv_agent.run(query)

In [ ]:
# 🧠 Recall Last Interaction
def recall_last_interaction(memory):
    messages = memory.chat_memory.messages
    if len(messages) >= 2:
        user_msg = messages[-2].content
        ai_msg = messages[-1].content
        return f"🧠 Last Question: {user_msg}\n💬 Last Answer: {ai_msg}"
    return "🧠 No previous memory found."


In [ ]:
def show_full_chat(memory):
    print("🧠 Chat History:")
    for msg in memory.chat_memory.messages:
        role = msg.type.capitalize()
        print(f"{role}: {msg.content}")

In [ ]:
print(master_agent("What is the sum of the ending balance of the ending balance in global currency? Explain how you arrived at the answer."))

In [ ]:
print(recall_last_interaction(finance_sql_memory))

In [ ]:
# agent_executer.invoke("How many total company codes are there in the dataset? Name them all.")

In [ ]:
# agent_executer.invoke("What is the sum of the debit balance in global currency? Explain how you arrived at the answer.")

In [ ]:
# agent_executer.invoke("What is the sum of the total credit balance in global currency? Explain how you arrived at the answer.")

In [ ]:
# agent_executer.invoke("Give me a list of the top 3 companies with the highest total credit balance in global currency. Explain how you arrived at the answer.")

In [ ]:
# agent_executer.invoke("What is the sum of the ending balance of the ending balance in global currency? Explain how you arrived at the answer.")

In [ ]:
# master_agent("What is the sum of the ending balance of the ending balance in global currency? Explain how you arrived at the answer.")

In [ ]:
# master_agent("What was the question I asked last and what was your answer?")